In [ ]:
import torch
import numpy as np
from tqdm.notebook import tqdm

from datasets import load_dataset
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

from bert_utils import TextSentiment, compute_centers_cov, mahalanobis_score, metrics_eval

imdb = load_dataset('imdb')
yelp = load_dataset("yelp_polarity")
tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-cased')

trainset = TextSentiment(imdb['train'], tokenizer, finetune=True)
testset = TextSentiment(imdb['test'], tokenizer, finetune=True)

model = torch.hub.load('huggingface/pytorch-transformers',
                       'modelForSequenceClassification', 
                       'bert-base-cased',
                       output_attentions=False).cuda()
testloader = torch.utils.data.DataLoader(testset, batch_size=1,shuffle=True,  num_workers=2)

In [ ]:
# This cell's code is adapted from HuggingFace library documentation

training_args = TrainingArguments(
    output_dir='./models/bert/results',          # output directory
    num_train_epochs=10,                         # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./models/bert/logs',            # directory for storing logs
    logging_steps=10,
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=trainset,              # training dataset
    eval_dataset=testset                 # evaluation dataset
)

trainer.train()

In [3]:
torch.save(model, './models/bert/IMDB_finetuned10')

In [20]:
testloader = torch.utils.data.DataLoader(testset, batch_size=16,shuffle=True,  num_workers=2)

def eval_acc(model, quantity_target=1000):
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for data in tqdm(testloader, total=quantity_target//testloader.batch_size):
            images, labels = data['input_ids'].cuda(), data['labels'].cuda()
            outputs = model(images)[0]
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted.view(-1,1) == labels).sum().item()
            if total>=quantity_target:
                break
    model.train()
    print(f'Validation accuracy: {100 * correct / total:.2f}')
    return(correct / total)
eval_acc(model)


Validation accuracy: 80.75


0.8075396825396826